In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import pandas as pd
import pandas as pd

In [7]:
door_data = pd.read_csv("./data/data/output/door/door_scratch_data.csv")

fender_data=pd.read_csv("./output/source_data/")

# 데이터 로딩
dataset = 

x = dataset['data']
y = dataset['target']

In [8]:
from sklearn.utils import resample
import numpy as np
from sklearn.model_selection import train_test_split


# data sampling
x_sampled, y_sampled = resample(x, y, n_samples=100, random_state=42)

print(x_sampled.shape)
print(y_sampled.shape)


# data split
x_training, x_test, y_training, y_test = train_test_split(x_sampled, y_sampled, test_size=0.2, shuffle=True, stratify=y_sampled, random_state=34)
x_train, x_valid, y_train, y_valid = train_test_split(x_training, y_training, test_size=0.2, shuffle=True, stratify=y_training, random_state=34)

(100, 4)
(100,)


In [9]:
# 모델 코드 작성

import torch.nn as nn
import torch.nn.functional as F


class ResNetLikeCNN(nn.Module):
    def __init__(self):
        super(ResNetLikeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # 3x3 Conv
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 10)  # Output layer (10 classes)

        self.pool = nn.MaxPool2d(2, 2)
        self.batchnorm1 = nn.BatchNorm2d(16)  # Batch Normalization
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm3 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNetLikeCNN()
model.to(device)

ResNetLikeCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [10]:
# 손실함수 정의
loss_fn = nn.CrossEntropyLoss()
# # 가중치 학습1
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 가중치 학습2
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
# 가중치 학습3
# optimizer = optim.SGD(model.parameters(), lr=0.01,
#                       momentum=0.9, weight_decay=5e-4)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
# 가중치 학습 4
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

In [11]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import torch

# 데이터셋 클래스 정의 (커스텀 데이터셋)

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


# 데이터 전처리 (Normalization 및 그레이스케일 변환)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=4),  # 그레이스케일로 변환 후 4채널로 확장
    transforms.RandomHorizontalFlip(),  # 데이터 증강 : 이미지 좌우 반전
    transforms.RandomCrop(32, padding=4),  # 데이터 증강 : padding 4 후 32*32로 랜덤 크롭
    transforms.ToTensor(),  # 텐서로 변환
    transforms.RandomRotation(10),  # 10도 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조정

    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])



# CustomDataset으로 데이터셋 만들기
train_dataset = CustomDataset(
    image_paths=x_training, labels=y_training, transform=transform)
# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(
    image_paths=x_valid, labels=y_valid, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [12]:
# # data split
# x_training, x_test, y_training, y_test = train_test_split(x_sampled, y_sampled, test_size=0.2, shuffle=True, stratify=y_sampled, random_state=34)
# x_train, x_valid, y_train, y_valid = train_test_split(x_training, y_training, test_size=0.2, shuffle=True, stratify=y_training, random_state=34)

In [ ]:
from PIL import Image, ImageFile, UnidentifiedImageError
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 손상된 이미지를 처리하도록 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 손상된 이미지가 있으면 건너뛰기


def safe_load_image(image_path):
    try:
        image = Image.open(image_path)
        image.load()  # 이미지가 제대로 로드되는지 확인
        return image
    except (OSError, UnidentifiedImageError, ValueError):
        print(f"Skipping corrupted image: {image_path}")
        return None  # 손상된 이미지는 None 반환

# 데이터셋 클래스 정의 (커스텀 데이터셋)


class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.transform = transform
        self.data = []

        for img_path, label in zip(image_paths, labels):
            # 이미지 로드 및 유효성 확인
            image = safe_load_image(img_path)
            if image is not None:
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]

        # 이미지를 다시 안전하게 로드
        image = safe_load_image(image_path)
        if self.transform:
            image = self.transform(image)

        return image, label


losses = []
# 모델 학습
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):  # train_loader에서 가져오기
        # 손상된 이미지가 반환된 경우 건너뛰기
        if inputs is None:
            continue

        # 입력 데이터와 레이블을 device로 이동
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)  # 모델에 입력
        losses = loss_fn(outputs, labels)
        losses.backward()
        optimizer.step()
        running_loss += losses.item()

    # 학습률 스케줄러 업데이트
    # scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

print("학습 완료")

AttributeError: 'numpy.ndarray' object has no attribute 'read'

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

In [ ]:
## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
pred=model(y_test)


# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat)


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도: ', accuracy)


# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred)
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred)
print('재현율: ', recall)


# roc auc
fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

plt.plot(fpr, tpr)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

plt.show()

print('auc 점수: ', roc_auc_score(y_test, pred))

In [ ]:
# 적용 예시


